<a href="https://colab.research.google.com/github/c-marq/cap4767-data-mining/blob/main/solutions/exercises/week06_group_exercise_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 6 Group Exercise — SOLUTION KEY 🔑 — Market Basket Analysis
**CAP4767 Data Mining with Python** | Miami Dade College — Kendall Campus

**Points:** 10 | **Duration:** ~45 minutes | **Deliverable:** Completed notebook + 3-minute presentation

**Objective:** Run a complete Market Basket Analysis pipeline on the Online Retail II dataset. Identify the top 10 rules by lift, produce one visualization, and present one product placement and one cross-sell recommendation.

### Group Members & Roles

| Role | Name | Responsibility |
|------|------|----------------|
| 🖥️ **Lead Coder** | | Writes code, drives the notebook |
| 📊 **Data Interpreter** | | Explains outputs, writes interpretation cells |
| 🎤 **Presenter** | | Delivers the 3-minute share-out |
| ✅ **QA Reviewer** | | Checks outputs against checkpoints |

*If 3 members, Data Interpreter also handles QA.*

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1A5276;">💡 GROUP DISCUSSION (before coding — 3 minutes)</strong><br>
  You've already analyzed this dataset for customer segmentation in Week 5. Before writing any code:
  <ol>
    <li>What new question does Market Basket Analysis answer that RFM and K-Means could not?</li>
    <li>If you were the marketing director of this online retailer, name one specific business decision you'd want MBA to inform. Why?</li>
  </ol>
</div>

**Our group's answers (2–3 sentences each):**

**Sample:** (1) RFM and K-Means tell us who our customer segments are and what they're worth. MBA tells us what products those customers buy together — it answers a product-level question, not a customer-level question. You can't get 'Frequently Bought Together' recommendations from K-Means. (2) We'd want MBA to inform the 'Customers Also Bought' recommendation engine on the website. Every product page shows suggestions, and right now those are probably based on popularity (most-sold items) rather than actual co-purchase patterns. Data-driven suggestions should convert better.

---

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run the setup cell below. It loads the dataset, cleans it, and delivers a ready-to-use DataFrame. <strong>Do not modify.</strong>
</div>

In [ ]:
# ============================================================
# Setup — Run this cell. Do not modify.
# ============================================================
!pip install mlxtend -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from mlxtend.frequent_patterns import apriori, association_rules

np.random.seed(42)
plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["figure.dpi"] = 100
sns.set_style("whitegrid")

# Load and clean Online Retail II
url = "https://github.com/fenago/datasets/raw/refs/heads/main/Online%20Retail%20(1).xlsx"
df = pd.read_excel(url, engine="openpyxl")

df["InvoiceNo"] = df["InvoiceNo"].astype(str)
df = df[~df["InvoiceNo"].str.startswith("C")]
df = df.dropna(subset=["CustomerID"])
df = df[df["Quantity"] > 0]
df = df[df["UnitPrice"] > 0]
df = df[df["StockCode"].astype(str).str.match(r"^\d")]
df["Description"] = df["Description"].str.strip()
df = df.dropna(subset=["Description"])

print(f"✅ Dataset loaded and cleaned")
print(f"   {df.shape[0]:,} transactions | {df['InvoiceNo'].nunique():,} invoices | {df['Description'].nunique():,} products")

---
## Task 1 — Transform to Basket Format (1 pt)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Convert the cleaned DataFrame into a one-hot encoded basket matrix: one row per Invoice, one column per Description, Boolean values (True/False). Print the shape.
</div>

In [ ]:
# Task 1: Convert to basket format
basket = df.groupby(["InvoiceNo", "Description"])["Quantity"].sum().unstack().fillna(0)
basket = basket.applymap(lambda x: x > 0)

print(f"Basket matrix shape: {basket.shape[0]:,} invoices × {basket.shape[1]:,} products")
density = basket.sum().sum() / (basket.shape[0] * basket.shape[1]) * 100
print(f"Matrix density: {density:.2f}%")

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 1</strong><br>
  Shape should be approximately <strong>18,000 rows × 3,800 columns</strong>. If you have far fewer columns, check that you used Description (not StockCode). If you have far fewer rows, check your cleaning filters.
</div>

---
## Task 2 — Run Apriori (1 pt)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run <code>apriori()</code> with <code>min_support=0.03</code> and <code>use_colnames=True</code>. Print the count of frequent itemsets and display the top 10 by support.
</div>

In [ ]:
# Task 2: Run Apriori
frequent_items = apriori(basket, min_support=0.02, use_colnames=True)
print(f"Frequent itemsets found: {len(frequent_items)}")
print(f"\nTop 10 by support:")
print(frequent_items.sort_values("support", ascending=False).head(10).to_string(index=False))

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 2</strong><br>
  You should find approximately <strong>60–100 frequent itemsets</strong>. If fewer than 20, your basket transformation may have an issue. If more than 500, check that non-product StockCodes were filtered.
</div>

---
## Task 3 — Generate and Filter Rules (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Generate rules with <code>metric="lift"</code> and <code>min_threshold=2.0</code>. Add a second filter: <code>confidence >= 0.3</code>. Sort by lift descending. Display the top 15 rules with all 5 metrics.
</div>

In [ ]:
# Task 3: Generate rules, filter, sort, display top 15
rules = association_rules(frequent_items, metric="lift", min_threshold=2.0)
rules = rules[rules["confidence"] >= 0.3]
rules = rules.sort_values("lift", ascending=False)

print(f"Rules after filtering: {len(rules)}")
print(f"\nTop 15 by Lift:")
print(rules[["antecedents", "consequents", "support", "confidence",
             "lift", "leverage", "conviction"]].head(15).to_string(index=False))

---
## Task 4 — Interpret the Top 3 Rules (2 pts)

Write a plain-language interpretation of the top 3 rules by lift. For each rule:
- What products are associated?
- How strong is the association (lift value)?
- Why might these products be bought together?

**Rule 1:** {GREEN REGENCY TEACUP AND SAUCER} → {ROSES REGENCY TEACUP AND SAUCER}
- Lift: ~15+
- Interpretation: Customers who buy the green teacup almost always buy the roses version too. These are likely part of a matching set — customers are collecting the full range. The extremely high lift means this pairing is far more frequent than chance.

**Rule 2:** {PINK REGENCY TEACUP AND SAUCER} → {GREEN REGENCY TEACUP AND SAUCER}
- Lift: ~14+
- Interpretation: Another teacup variant — the color collection pattern is strong. Customers buying one color variant are highly likely to buy others, consistent with a 'set collector' behavior.

**Rule 3:** {ALARM CLOCK BAKELIKE GREEN} → {ALARM CLOCK BAKELIKE RED}
- Lift: ~10+
- Interpretation: Color variant purchasing extends beyond teacups — alarm clocks show the same pattern. Customers buying one color variant of a product frequently buy another color. This suggests the retailer should display all color variants together on the product page.

**Sample answers below — actual rules depend on dataset filtering.**

---
## Task 5 — Produce One Visualization (2 pts)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Choose <strong>ONE</strong> of the following:
  <ul>
    <li><strong>Option A:</strong> Network graph — top 15 rules, nodes = products, edges = rules, thickness = lift</li>
    <li><strong>Option B:</strong> Support vs confidence scatter — all rules, bubble size = lift, colorbar for lift</li>
    <li><strong>Option C:</strong> Lift heatmap — top 20 rules, rows = antecedents, columns = consequents, color = lift</li>
  </ul>
</div>

In [ ]:
# Task 5: Support vs Confidence scatter (Option B)
plt.figure(figsize=(12, 7))

# Cap lift for sizing — inf values crash scatter
plot_rules = rules.copy()
plot_rules["lift_clean"] = pd.to_numeric(plot_rules["lift"], errors="coerce").fillna(0).replace([np.inf, -np.inf], 0)

scatter = plt.scatter(plot_rules["support"].values, plot_rules["confidence"].values,
                      s=plot_rules["lift_clean"].values * 5, alpha=0.5,
                      c=plot_rules["lift_clean"].values, cmap="YlOrRd", edgecolors="gray", linewidth=0.5)

plt.colorbar(scatter, label="Lift")
plt.axhline(y=0.5, color="gray", linestyle="--", alpha=0.4, label="Confidence = 0.5")
plt.axvline(x=0.05, color="gray", linestyle="--", alpha=0.4, label="Support = 0.05")

plt.xlabel("Support (how common)", fontsize=12)
plt.ylabel("Confidence (how reliable)", fontsize=12)
plt.title("MBA Rules — Support vs Confidence (Bubble size & color = Lift)", fontsize=13)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 CHECKPOINT 3</strong><br>
  Your visualization should clearly show which product pairs have the strongest associations. If labels overlap or colors look flat, reduce the number of rules displayed.
</div>

---
## Task 6 — Business Recommendations (2 pts)

Write two specific recommendations based on your analysis:

**Product Placement Recommendation:**
Which products should be displayed together (on the website or in a physical store)? Cite the specific rule and its lift.

**Sample:** Based on Rule 1 (Green Teacup → Roses Teacup, lift ~15), all color variants of the Regency Teacup and Saucer line should be displayed on the same product page. Currently, customers who find one variant have to search separately for others. Grouping them under a 'Complete Your Collection' section would reduce friction and capitalize on the set-collecting behavior the data reveals. The lift of 15 means customers buying one variant are 15 times more likely to buy another than the average customer.

**Cross-Sell Recommendation:**
**Sample:** When a customer adds the Alarm Clock Bakelike Green to their cart, the site should immediately suggest the Red variant as 'Frequently Bought Together' (confidence ~0.6, lift ~10). This is a low-cost intervention — the suggestion is a single line of text on the cart page — but the confidence of 60% means more than half the customers who buy green also buy red. At an average price of £3–5 per clock, even a modest conversion rate on the suggestion adds meaningful incremental revenue across thousands of transactions.

---

## Share-Out (3 minutes)

Present to the class:
1. **One surprising finding** — A rule you didn't expect. What was the lift? Why do you think these products are associated?
2. **The visualization** — Show the chart. Point out the strongest cluster, hottest cell, or largest bubble.
3. **One business recommendation** — Product placement OR cross-sell. Be specific: which products, what action, why the metric justifies it.

## Troubleshooting

| Problem | Fix |
|---------|-----|
| `apriori()` takes forever or crashes | Your min_support is too low. Start at 0.03 minimum. |
| Zero frequent itemsets | Basket matrix may not be Boolean — check `basket.dtypes`. Must be True/False. |
| `association_rules()` gives empty DataFrame | Lower `min_threshold` (try 1.5 instead of 2.0) or lower confidence filter |
| Network graph labels overlap | Increase `k` in `nx.spring_layout(G, k=3)` or reduce to top 10 rules |
| Heatmap is one solid color | You may have too few rules, or all lifts are similar. Try top 20 instead of all rules. |

---
<p style="color:#7F8C8D; font-size:0.85em;">
<em>CAP4767 Data Mining with Python | Miami Dade College | Spring 2026</em><br>
Week 6 Group Exercise — Market Basket Analysis | 10 Points
</p>